# Game theory

### Theory Recap

A **game** in strategic form is a triplet $\left(\mathcal V, \mathcal A, \{u_i\}_{i \in \mathcal V}\right)$ where:
- $\mathcal V$ is the finite set of **players**
- $\mathcal A$ is the set of **actions** available to each player. Actions $x_i \in \mathcal A$ chosen  by each player are collected in a vector $x = (x_1,x_2, \ldots, x_n)$ called **configuration** of the game. We denote the set of all configurations by $\mathcal X = \mathcal A^{\mathcal V}$.
- for each player $i \in \mathcal V$, the utility function $u_i: \mathcal X \rightarrow \mathbb{R}$ describes the **payoff** the player gets in each configuration: the utility of a player depends on its own action but also on the actions of other players.	

To each player $i \in \mathcal V$ we associate a **best response** function, which gives, for each configuration of the other players, the best action for $i$ (i.e., the actions that maximize its utility). Let $x_{-i}$ denote the vector strategies played by all players but $i$, and let $\mathcal B$ denote the best response function, i.e.,

$$
\mathcal B_i(x_{-i})= \arg\max_{x_i \in \mathcal A} u_i(x_i,x_{-i})
$$

A (pure strategy) **Nash equilibrium** is a configuration $x^* \in \mathcal X$ such that

$$
x_i^* \in \mathcal B_i (x^*_{-i}), \quad \forall i \in \mathcal V
$$

A game is **potential** if there exists a potential function $\Phi: \mathcal X \rightarrow \mathbb{R}$ which describes the way the utility function of each player changes when that player unilaterally varies its action. That is, $\forall i \in \mathcal V$, $\forall x,y \in \mathcal X$

$$
x_{-i}=y_{-i} \implies u_i(y)-u_i(x) = \Phi(y)-\Phi(x)
$$

# Discrete time best response dynamics
In this section we present and analyze a first game-theoretic learning process, the discrete time best response dynamics.

Consider a game $\left(\mathcal V, \mathcal A, \{u_i\}_{i \in \mathcal V}\right)$. 

The **discrete-time asynchronous best response dynamics** is a Markov chain $X(t)$ with state space $\mathcal X= \mathcal A^{\mathcal V}$ coinciding with the configuration space of the game.
At time instant $t$, when the chain is in $X(t)$, one player $i$ is selected uniformly at random in $\mathcal V$ and she changes her action to a best response chosen uniformly at random in the set $B_i(X(t)_{-i})$.

In other words, the best response dynamics is random walk on the transition graph $\mathcal G_{B}$, i.e., the graph with nodes corresponding to configurations $\mathcal X=\mathcal A^{\mathcal V}$ and links $(x,y)$ from configurations $x$ and $y$ that differ in exactly one entry $i$ if and only if $y_i \in B_i(x_{-i})$.

## Example: Rock-Scissor-Paper game
The Rock-Scissor-Paper game is a two-player symmetric game with action set $\mathcal A =\{R, S, P\}$ and utility matrix

![utility-matrix](RSP-utility.png)

1. Write a function that implements the best response correspondence for both players.
2. Does the game admit Nash equilibria?
3. Is the game potential?
4. Consider the best response dynamics for the RSP game. Plot the transition graph $\mathcal G_{B}$.
5. Simulate the (discrete time) best response dynamics and estimate the invariant probability distribution $\pi$.
6. Compute and plot the condensation graph of $\mathcal G_{B}$.
7. Compute analytically the invariant distribution $\pi$ of the best response dynamics.


> 1. Write a function that implements the best response for both players.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define action vector
# we identify actions with numbers: R->0, S->1, P->2
actions = np.array([0,1,2])

# Define utility matrix
# first index is the player whose utility is being computed 0=first/row player,
# 1=second/column player,
# second index is the first player action 0=R, 1=S, 2=P,
# third index is the second player action 0=R, 1=S, 2=P
utility = np.array([[[0, 1, -1],
                     [-1, 0, 1],
                     [1, -1, 0]],
                    [[0, -1, 1],
                     [1, 0, -1],
                     [-1, 1, 0]]
                   ])

print(utility)


# Best response function for 2 player games 
def best_response(player, configuration):
    # if player == 1, other_player = 0 (and viceversa)
    other_player = 1-player
    # other action is the action of the other player
    other_action = configuration[other_player]
    if player == 0:
        # if player == 0, player 0 looks at the action of player 1 (other_action) and chooses BR (2nd index)
        return [np.argmax(utility[player, :, other_action])]
    if player == 1:
        # if player == 1, player 1 looks at the action of player 0 (other_action) and chooses BR (3nd index)
        return [np.argmax(utility[player, other_action, :])]

> 2. Does the game admit Nash equilibria?

In [ ]:
from itertools import product

# Compute all game's configurations
# product function computes the cartesian product of actions with itself
# returns an iterator of tuples of actions
configurations = list(product(actions, repeat=2))
print("Configurations:", configurations, "\n")

nash_list = []

# we cycle over configurations:
# each configuration is a Nash unless there exists (at least) one player who
# is not playing a best response to the configuration of other players
for configuration in configurations:
    is_nash = True
    for player in [0,1]:
        if configuration[player] not in best_response(player,configuration):
            is_nash = False
            break
    if is_nash:
        nash_list.append(configuration)
        
if len(nash_list) != 0:
    print("The game admits the following Nash equilibria", nash_list)
else:
    print("The game does not posses Nash equilibria.")

The game does not posses any Nash equilibrium. In each configuration at least one player has an incentive in changing is current action to a best response, which gives him utility $+1$. There are no configurations where both players have utility $+1$.

> 3. Is the game potential?

If a finite game (i.e., a  game with a finite configuration set) is potential, then the set of its Nash equilibria is non-empty as it contains the maxima of the potential function $\Phi$ (at least a maximum exists due to Weierstrass' theorem).
The RSP game does not posses Nash equilibria, so it is not potential.

> 4. Consider the best response dynamics for the RSP game. Plot the transition graph $\mathcal G_{B}$ (i.e., the graph with nodes corresponding to configurations $\mathcal X=\mathcal A^{\mathcal V}$ and links $(x,y)$ between configurations $x$ and $y$ that differ in exactly one entry $i$ if and only if $y_i \in B_i(x_{-i})$).

In [ ]:
import networkx as nx

# transition_graph is a directed graph
transition_graph = nx.DiGraph()
# nodes of transition_graph are configurations
transition_graph.add_nodes_from(configurations)

# we visit each configuration x and add links to neighboring configurations.
# Neighboring configurations y of x are those that differ in exactly one entry,
# the action of a single player i, and such that y_i is a best response to
# x_i for player i
for configuration in configurations:
    for player in [0,1]:
        for other_action in actions:
            # selects other action in best response, but no selfloops
            if other_action != configuration[player] and other_action in best_response(player,configuration):
                other_config = list(configuration)
                other_config[player] = other_action
                other_config = tuple(other_config)
                transition_graph.add_edge(configuration,other_config)
                
pos = nx.spring_layout(transition_graph)
nx.draw(transition_graph, pos=pos, with_labels=True)       

The transition graph has nodes corresponding to the game configurations, and directed links which represent admissible transitions for the best response dynamics. 

> 5. Simulate the (discrete time) best response dynamics and estimate the invariant probability distribution $\pi$.

In [ ]:
from numpy.random import choice, rand 

# Simulates the best response dynamics as a random walk on the transition graph G
# starting from node xi.
# if till_first_return = True the random walk stops the first time
# it returns to the starting node xi.
# Otherwise, it goes on for num_steps steps.

def RandomWalk(G, xi, num_steps, till_first_return = False):
    # nodeSeq stores the sequence of visited nodes
    nodeSeq = []
    nodeSeq.append(xi)
    
    # if the walk ends at the first return to xi
    if till_first_return:
        # stores the initial position to check if the 
        # walk returns to it
        x_init = xi
        
        while True:
            # compute the next visited node xi by chosing uniformly
            # at random a neighbor of the current one
            neighbors = list(G.neighbors(xi))
            index = choice(len(neighbors),1)[0] 
            xi = neighbors[index] 
            nodeSeq.append(xi)
            
            # check if the walk has returned to the starting node
            # if so, end the walk
            if xi == x_init:
                return nodeSeq
    
    # if the walk ends after num_steps steps
    else:
        for i in range(num_steps):
            neighbors = list(G.neighbors(xi))
            index = choice(len(neighbors),1)[0] 
            xi = neighbors[index]
            nodeSeq.append(xi)
        return nodeSeq

# Simulate the BR dynamics 
nodeSeq = RandomWalk(transition_graph, xi=(0,0), num_steps=100, till_first_return=False)
edgeSeq = [(nodeSeq[i-1], nodeSeq[i]) for i in range(1,len(nodeSeq))]

# Draw G and represent the random walk by colouring the edge sequence
# first draw all nodes and links
nx.draw(transition_graph, pos)
# then, on the previous picture, add node labels and highlight the edge sequence
nx.draw(transition_graph, pos, with_labels=True, edgelist = edgeSeq, edge_color='blue', node_color='red', width=2)

We see that the dynamics starts at configuration $(R,R)$ (represented by $(0,0)$) and after one step it leaves it. Then it continues cycling over best response cycle $(0,2) \to (1,2) \to (1,0) \to (2,0) \to (2,1) \to (0,1) \to (0,2)$. In particular, it never goes back to $(0,0)$ and $(1,1),(2,2)$ are never visited.

In [ ]:
# Estimate the invariant measure pi of the BR dynamics by 
# simulating a "long" random walk

nodeSeq = RandomWalk(transition_graph, (0,0), 10000, False)

# Compute empirical frequencies

# Define a dictionary to store frequencies of visit to each node and initialize each frequency to 0
frequencies = {node:0 for node in transition_graph.nodes}
# count the visits to each node
for node in nodeSeq:
    frequencies[node] += 1
# transform dict values to a np.array
frequencies = list(frequencies[node] for node in transition_graph.nodes) 
frequencies = np.array(frequencies, dtype="double")
# normalize the counts to obtain frequencies
frequencies /= len(nodeSeq)
print("Frequencies:", frequencies, "\n")

# Print the indices of nodes where pi is positive
print("The approximate invariant distribution is supported on nodes:")
for index,node in enumerate(transition_graph.nodes):
    if frequencies[index]>0:
        print(node)

The approximate invariant distribution appears to be supported on nodes $\{(0,0),(0,1),(0,2),(1,0),(1,2),(2,0),(2,1)\}$ of the transition graph. However, the measure of node $(0,0)$ is negligible with respect to that of $(0,1),(0,2),(1,0),(1,2),(2,0),(2,1)$, which are almost equal to each other. Actually, the little weight associated to node $(0,0)$ appears to depend on the fact that it has been visited once since it is the starting point of the chain. We guess that when the length on the walk tends to infinity the weight associated to node $(0,0)$ vanishes and the approximation of $\pi$ is supported and uniformly distributed on $\{(0,1),(0,2),(1,0),(1,2),(2,0),(2,1)\}$.

> 6. Compute and plot the condensation graph of $\mathcal G_{B}$.

In [ ]:
CG = nx.algorithms.components.condensation(transition_graph)
nx.draw(CG, with_labels=True)
print(dict(CG.nodes))

The condensation graph is shown above, where the hypernode $0$ contains all configurations but $(R,R), (S,S), (P,P)$ (the ones that correspond to a draw outcome). More precisely, the hypernode $0$ contains the $6$-cycle $(0,2) \to (1,2) \to (1,0) \to (2,0) \to (2,1) \to (0,1) \to (0,2)$. The hypernodes $1,2,3$ instead contain the singletons $(R,R), (S,S), (P,P)$. The condensation graph contains only one sink, that is $s_{\text{transition_graph}} = 1$.

7. Compute analytically the invariant distribution $\pi$ of the best response dynamics.

We know that the invariant probability distribution is supported on the nodes contained in the unique sink of the condensation graph. The subgraph corresponding to such sink is balanced (all nodes have in-degree $1$ and out-degree $1$, so the sink is $1$-regular), then the extremal invariant probability distribution supported on that sink is obtained by normalizing the out-degree vector. In this way we find that $\pi_x = 1/6$, for $x \in \{RS, RP, SR, SP, PR, PS\}$ and $\pi_x = 0$ for $x \in \{RR, SS, PP\}$.

We can check that this is correct with the following code.

In [ ]:
# Compute P matrix
A = nx.adjacency_matrix(transition_graph) # -> return type is scipy.sparse.csr_matrix
A = A.toarray() # convert A to a numpy array
degrees = np.sum(A,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ A

# Compute invariant distribution
values,vectors = np.linalg.eig(P.T)
index = np.argmax(values.real)
pi = vectors[:,index].real
pi = pi/np.sum(pi)
print("pi=", pi)

## When a game is potential

In previous exercise, it was asked whether the RSP is a potential game. To answer this question, we used the fact that every potential game admits at least a (pure strategy) Nash equilibrium. Since this is not the case for RSP, the game is not potential. An alternative way to address this question is to use the following characterization.

**Theorem**: consider a game $u =\left(\mathcal V, \mathcal A, \{u_i\}_{i \in \mathcal V}\right)$ and a finite path $\gamma = (x_0, x_1, \ldots, x_N)$ in the configuration graph of the game. Let

$$
I(\gamma, u)= \sum_{k=1}^{N}[u_{i_k}(x_k) - u_{i_k}(x_{k-1})]
$$

where $i_k$ is the only player that modifies her action at time-step $k$, i.e., $(x_k)_{i_k} \neq (x_{k-1})_{i_k}$ e $(x_k)_{-i_k} = (x_{k-1})_{-i_k}$.The following statements are equivalent:
- a game is potential.
- $I(\gamma, u)=0$ for all finite closed path in the configuration graph.
- $I(\gamma, u)=0$ for all simple closed path with length 4 in the configuration graph.

**Exercise**: we consider a 2-players game with $\mathcal{A} = \{0,1\}$. The payoffs are in the table below.

![utility-matrix2](game22.png)

For what parameters the game is potential?

**Solution**: consider the simple path $\gamma_1 = ((0,0),(1,0),(1,1),(0,1),(0,0))$.

Along this path, 

$$I(\gamma_1,u) = (2-0) + (b-a) + (c-4) + (0-(-1)) = -1-a+b+c.$$

It is easy to observe that for every other closed path $\gamma_i$ with length 4,

$$I(\gamma_i,u) = I(\gamma_1,u) \quad \text{or} \quad I(\gamma_i,u) = -I(\gamma_1,u),$$

thus we can apply the theorem to state that the game is potential if and only if

$$
I(\gamma_1,u)=-1-a+b+c = 0.
$$

Let us consider for example the case $a=c=0, b=1$, under which the game is potential. The game is

|   | 0   | 1   |
|---|-----|-----|
| **0** | 0,0 | 0,-1 |
| **1** | 2,0 | 4,1 |

It is easy to see that $(1,1)$ is a Nash equilibrium for the game.

If one considers instead the case $c=5, a=-4, b=-2$, which does not satisfy the sufficient and necessary condition for the game being potential.

|   | 0   | 1   |
|---|-----|-----|
| **0** | 0,0 | 5,-1 |
| **1** | 2,-4 | 4,-2 |

The game is not potential, as proved by the fact that it does not admit Nash equilibria.

Still, it may be that a game with Nash equilibria is not potential, as for instance $c=-2, a=-4, b=-2$.

|   | 0   | 1   |
|---|-----|-----|
| **0** | 0,0 | -2,-1 |
| **1** | 2,-4 | 4,-2 |

For this game indeed $(1,1)$ is a Nash equilibrium

## Continuous time best response dynamics
In this section we present the continuous time best response dynamics, which is the continuous time analog of the process presented in the previous section.

Consider a game $\left(\mathcal V, \mathcal A, \{u_i\}_{i \in \mathcal V}\right)$. 

The **continuous-time asynchronous best response dynamics** is a continuous time Markov chain $X(t)$ with state space $\mathcal X= \mathcal A^{\mathcal V}$ coinciding with the configuration space of the game and transition rate matrix $\Lambda$ as follows: 

$\Lambda_{xy} = 0$ for every two configurations $x, y \in \mathcal X$ that differ in more
than one entry, and

$$
\Lambda_{xy} = \begin{cases}
|B_i(x_{-i})|^{-1} \quad &\text{if} \quad y_i \in B_i(x_{-i}) \\
0 \quad &\text{if} \quad y_i \notin B_i(x_{-i})
\end{cases}
$$

for every two configurations $x, y \in \mathcal X$ differing in entry $i$ only, i.e., such that
$x_i \neq y_i$ and $x_{-i} = y_{-i}$.

## Continuous-time best response dynamics in potential games

**Proposition**: consider a potential game, and let $\mathcal{N}$ the set of the Nash equilibria. Then, for every distribution of initial configuration $X(0)$, the best response dynamics converges to $\mathcal{N}$ in finite time with probability 1. 

**Remark**: Convergence is in fact insured to a particular subset of NE, that is the one consisting of the nodes of all the sink connected components of the underlying transition graph of the Markov process. In the sequel we denote such subset as $\mathcal{N}_\infty$: it is a trapping set and actually the largest trapping set inside $\mathcal{N}$. We call $\mathcal{N}_\infty$ the set of **recurrent Nash equilibria**. Notice that in those special cases when $\mathcal{N}_\infty$ coincides with $\text{argmax}(\Phi(x))$, we actually have that the BR dynamics converges to the subset of Nash equilibria consisting of the maxima of the potential. However, this is not always the case.

The continuous-time best response dynamics has a similar behaviour to discrete-time asynchronous best-response. We do not implement here the continuous-time best response dynamics. We instead focus on **continuous-time noisy best reponse dynamics**.

## Continuous-time noisy best response dynamics

We consider a **continuous time asynchronous noisy best response  dynamics** with inverse noise parameter $\eta$, i.e., a continuous time Markov chain on the configuration space $\mathcal X$ (the set of all game's configurations) with transition rates $\Lambda_{xy} = 0$ for every two configurations $x, y \in \mathcal X$ that differ in more
than one entry, and

$$
\Lambda_{xy} = \frac{\exp^{\eta u_i(y_i,x_{-i})}}{\sum_{a \in \mathcal A} \exp^{\eta u_i(a,x_{-i})} }
$$

for every two configurations $x, y \in \mathcal X$ differing in entry $i$ only, i.e., such that
$x_i \neq y_i$ and $x_{-i} = y_{-i}$.

Note that the probability that a player chooses a certain strategy $x_i$ is increasing in the payoff associated to that strategy. In particular,

- if $\eta=0$, the players choose strategies independently of the associated payoff (infinite noise, the game basically disappears), i.e., for every two configurations $x, y \in \mathcal X$ differing in entry $i$ only, i.e., such that
$x_i \neq y_i$ and $x_{-i} = y_{-i}$,

$$
\Lambda_{xy} = \frac{1}{|\mathcal{A}|}
$$

- if $\eta \to + \infty$, then the players randomize among optimal strategies, and assign probability $0$ to suboptimal ones (similar to best-response).

## Continuous-time noisy best response dynamics in potential games

The transition graph of the NBR is strongly connected for each value of $\eta$, i.e., from every configuration of strategies there is a non-null probability to reach any other configuration of strategies. In particular, for potential games, the NBR admits an invariant distribution

$$
\pi_x = \frac{e^{\eta \Phi(x)}}{Z_\eta}, \quad Z_\eta = \sum_{y\in \mathcal{X}} e^{\eta \Phi(y)}.
\tag{1}
$$

This is in contrast with best-response dynamics, which in potential games admits a trapping set of Nash equilibria $\mathcal{N}_\infty$ and converge to this set with probability 1 in finite time.

**Remark** Note that if $\eta \to +\infty$, the dynamics spends almost all the time in the global maximizers of the potential. However, global maximizers of the potential differ in general from $\mathcal{N}_\infty$ as defined above for best response dynamics.

## Network games

Network games are games where players are associated to nodes of a graph $\mathcal G$ that describes the interactions among them. In particular, for each player $i\in\mathcal V$ and for each couple of configurations $x,y\in\mathcal A^{\mathcal V}$ such that $x_j=y_j$ for each $j\in\ N_i\cup\{i\}$ it holds that

$$u_i(x)=u_i(y)$$

This means that the utility of each player only depends on the actions of its neighbors in the graph $\mathcal G$.


Given an unweighted undirected graph $\mathcal G=(\mathcal V,\mathcal E,W)$ we can construct a network game on it by setting the players’ utilities to coincide with the weighted sum of the payoffs of the same symmetric two-player game played by the player simultaneously with her neighbors.

Specifically, for a symmetric two-player game with action set $\mathcal A$ and utility function $\varphi(x_1, x_2)$ we define the network game $(\mathcal V, \mathcal A, \{u_i\}_{i \in \mathcal V} )$ by setting the utility of every player $i \in \mathcal V$ as
$$
u_i(x) = \sum_{j \in \mathcal V} W_{ij}\varphi(x_i,x_j)
$$

## Exercise - continuous time NBRD for network coordination game
A binary coordination game is a symmetric $2 \times 2$-game with action set $\mathcal A = \{0,1\}$ with payoff matrix $\varphi$

|   | 0   | 1   |
|---|-----|-----|
| **0** | a,a | d,c |
| **1** | c,d | b,b |

where $a > c$ and $b > d$. The inequalities above imply that the best response for each player is to copy the action of the other player.

It is a potential game with potential $\phi$

|   | 0   | 1   |
|---|-----|-----|
| **0** | a-c | 0 |
| **1** | 0 | b-d |

Consider the **network coordination game**, obtained combining binary coordination games on the link of the following undirected graph

![network](network.png)

Recall that such network game is potential with potential function $\Phi$

$$
\Phi(x) = \frac{1}{2} \sum_{i,j} W_{ij} \phi(x_i,x_j) .
$$

1.  Simulate the dynamics with $\eta=2$ when the coordination game has the same utility value for coordination of action 0 and action 1 respectively, i.e., $a=b=1$ and $c=d=0$. Run the simulation a few times and plot the potential function as a function of time. What do you observe?
2. Simulate the dynamics when  instead $a=1$, $b=\frac{1}{2}$ and $c=d=0$. Run the simulation a few times and plot the potential function as a function of time. What do you observe?
3. If the simulation continues for a long time, will the probability for different configurations converge towards some specific values? In that case, how can these be computed?
4. Compute the ratio between the probability that after a long time (i.e., in stationarity) all nodes choose action 1 and the probability that all nodes choose action 0, both for case **1** and case **2**. Does this seem to agree with your simulations?
5. Compute the ratio between the probability that after a long time (i.e., in stationarity) one node chooses action 1 and the rest choose action 0, and the probability that all nodes choose action 0, both for case **1** and case **2**.


### Solution
#### Step 1
> Simulate the dynamics when the coordination game has the same utility value for coordination of action 0 and action 1 respectively, i.e., $a=b=1$ and $c=d=0$. Run the simulation a few times and plot the potential function as a function of time. What do you observe?

In [ ]:
# Construct the undirected graph as shown in the picture
G = nx.Graph()
G.add_nodes_from(range(5))
G.add_edges_from([
    (0,1),(0,2),
    (1,2),(1,3),
    (2,3),(2,4),
    (3,4)
])

plt.figure(figsize=(5,7))
pos = {3: (0, 0), 4: (1, 0), 1: (0, 1), 2: (1, 1), 0: (0.5, 1.5)}
nx.draw(G, pos=pos, with_labels=True)

In [ ]:
# Define the network coordination game

# number of nodes in G = number of players in the game
n_players = len(G)
# utility values
a = 1
b = 1
c = 0
d = 0
# utility matrix for the 2x2 coordination game
phi = np.array([[a,d],
                [c,b]])
# the potential function of the 2x2 coordination game
pot = np.array([[a-c,0],
                [0,b-d]])
# inverse noise parameter
eta = 2
# available actions
actions = [0,1]
n_actions = len(actions)
# adjacency matrix
W = nx.convert_matrix.to_numpy_matrix(G)

def utility(player, x, phi):
    result = 0
    for other_player in G.neighbors(player):
         result += phi[x[player], x[other_player]]
    return result

In [ ]:
import scipy as sp

# Define the noisy best response dynamics

# Initialize transition rates matrix
n_config = n_actions**n_players
Lambda = sp.sparse.lil_matrix((n_config,n_config))

# Number of actions for each player
n_states =tuple(n_actions for _ in range(n_players))

# Fill transition rates matrix
for x_id in range(n_config):
    x = np.unravel_index(x_id,shape=n_states)
    x = np.array(x)
    for player in range(n_players):
        # compute utilities gained by `player` for each of its possible actions
        # while the other players are in the current configuration x
        utilities = np.zeros(n_actions)
        for action in actions:
            y = np.array(x)
            y[player] = action
            utilities[action] = utility(player,y,phi)
        # exp_utilities contains the exponential of utilities of 'player' for each possibile action
        exp_utilities = np.exp(eta*utilities)
        for action in actions:
            # there are no selfloops 
            if action == x[player]:
                continue
            y = np.array(x)
            y[player] = action
            y_id = np.ravel_multi_index(tuple(y), dims = n_states)
            # add a non-zero rate from x to y proportional to payoff of player when playing action (configuration is y)
            Lambda[x_id, y_id] += exp_utilities[action] / np.sum(exp_utilities)

In [ ]:
# Simulate the continuous time Markov chain with 2nd approach: local clocks
w = np.sum(Lambda, axis=1)

# reshape w
w = np.array(w.T)[0]
for x, weight in enumerate(w):
    # add selfloop if a configuration is a sink, otherwise D is not well defined
    if weight == 0:
        Lambda[x,x] = 1
        w[x] = 1
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# number of iterations
n_steps = 100

states = np.zeros(n_steps, dtype=int)
# initial configuration is random
x = np.random.choice(actions, size = n_players)
x_id = np.ravel_multi_index(tuple(x), dims = n_states)
states[0] = x_id

transition_times = np.zeros(n_steps)
t_next = -np.log(np.random.rand())/w[states[0]]

for i in range(1,n_steps):
    states[i] = np.random.choice(n_config, p=P[states[i-1],:])
    transition_times[i] = transition_times[i-1] + t_next
    t_next = -np.log(np.random.rand())/w[states[i]]

In [ ]:
# Compute and plot the evolution of the potential along the simulated 
# path of the chain

# store the potential value at each step of the simulation
potentials = np.zeros(n_steps)
for step in range(n_steps):
    x_id = states[step]
    x = np.array(np.unravel_index(x_id, shape = n_states))
    for i, j in G.edges:
        potentials[step] += pot[x[i],x[j]]

plt.step(transition_times, potentials, 'bo-', where="post");

In [ ]:
# To better understand the process, visualize the 
# evolution of the game's configuration in time

import matplotlib.pyplot as plt

# Plot the evolution of the game's configurations
fig = plt.figure(figsize=(9,12))
for t in range(0,min(n_steps,25)):
    plt.subplot(5,5,t+1)
    x_id = states[t]
    x = np.array(np.unravel_index(x_id, shape = n_states))
    nx.draw(G,
        pos = pos,
        with_labels=True,
        nodelist=np.argwhere(x==0).T[0].tolist(),
        node_color = 'r')
    nx.draw(G,
        pos = pos,
        with_labels=True,
        nodelist=np.argwhere(x==1).T[0].tolist(),
        node_color = 'b')
    plt.title('jump step = {0}'.format(t+1))

We can see that the potential during most of the
time is at its maximum value, corresponding to all nodes choosing the
same action. By repeating the simulation, it seems like all nodes tend
to end up in the same state after long time, independently of the initialization. The asymptotic state consists equally often in that all nodes
choose action 0 as that all nodes choose action 1.

We know from the theory that the noisy best response dynamics operates a selection among the NE and concentrates, in the small noise limit, most of the probability of its invariant distribution in the subset of those Nash equilibria that correspond to the set of potential maximizers 

$$\arg \max_{x \in \mathcal X} \Phi(x)$$ 

With the current choice for $a,b,c,d$ the maximum value of the potential $\Phi$ is realized by both the all-one and all-zero configurations.

In this example we have a fixed noisy parameter $\eta = 2$ but we are still able to observe that the dynamics spends most part of the time in global maxima of the potential. This can be evaluated for instance by computing the average of the potential function along the trajectory.

In [ ]:
intervals = np.diff(transition_times, n=1, append = transition_times[-1] + t_next)
avg_potential = np.sum(intervals*potentials)/(transition_times[-1] + t_next)
print("Average potential:", avg_potential)

Try to change the noise parameter and see how the dynamics behaves.

If $\eta=0$, then:
- the average potential is about 3.5, because transitions are random, the number of links is 7, and the potential counts the number of (undirected) links between nodes with same action. If the nodes choose randomly their state, in expectation the number of links between nodes with same action (and thus the potential) is 3.5.

If $\eta=5$, then:
- the dynamics spends most of the time in consensus configuration, thus the potential tends to 7, all the links connect nodes with same action;
- the average time between transitions increases, because it is unlikely that an agent modifies her action from a Nash equilibrium to achieve a lower payoff. Notice that the transition times increase only when the transition descends the potential.

#### Step 2
> Simulate the dynamics when  instead $a=1$, $b=\frac{1}{2}$ and $c=d=0$. Run the simulation a few times and plot the potential function as a function of time. What do you observe?

The same code as previously can be reused, by only changing the parameter $b$. The simulations and the evolution of the potential values along it behave very similarly, with the difference that, among the states in which all nodes choose the same action, state $(0,\ldots,0)$ is much more frequent than $(1,\ldots,1)$. Even if all nodes are initialized to choose action 1, with high probability after a while they will all choose action 0 instead. This happens since with the current choice of $a,b,c,d$ the global maximum of the potential $\Phi$ is realized by the all-zero configuration.

**Remark**: note however that, in contrast with the best response dynamics, the noisy best response dynamics still will escape from global maximizers of the potential ('all zeros' states in this example), because the transition graph is irreducible.

In [ ]:
# Define the network coordination game

# number of nodes in G = number of players in the game
n_players = len(G)
# utility values
a = 1
b = 1/2
c = 0
d = 0
# utility matrix for the 2x2 coordination game
phi = np.array([[a,d],
                [c,b]])
# the potential function of the 2x2 coordination game
pot = np.array([[a-c,0],
                [0,b-d]])
# inverse noise parameter
eta = 5
# available actions
actions = [0,1]
n_actions = len(actions)
# adjacency matrix
W = nx.convert_matrix.to_numpy_matrix(G)

def utility(player, x, phi):
    result = 0
    for other_player in G.neighbors(player):
         result += phi[x[player], x[other_player]]
    return result

In [ ]:
# Define the noisy best response dynamics

# Initialize transition rates matrix
n_config = n_actions**n_players
Lambda = sp.sparse.lil_matrix((n_config,n_config))

# Number of actions for each player
n_states =tuple(n_actions for _ in range(n_players))

# Fill transition rates matrix
for x_id in range(n_config):
    x = np.unravel_index(x_id,shape=n_states)
    x = np.array(x)
    for player in range(n_players):
        # compute utilities gained by `player` for each of its possible actions
        # while the other players are in the current configuration x
        utilities = np.zeros(n_actions)
        for action in actions:
            y = np.array(x)
            y[player] = action
            utilities[action] = utility(player,y,phi)
        exp_utilities = np.exp(eta*utilities)
        for action in actions:
            if action == x[player]:
                continue
            y = np.array(x)
            y[player] = action
            y_id = np.ravel_multi_index(tuple(y), dims = n_states)
            Lambda[x_id, y_id] += exp_utilities[action] / np.sum(exp_utilities)

In [ ]:
# Simulate the continuous time Markov chain with 2nd approach: local clocks

w = np.sum(Lambda, axis=1)
w = np.array(w.T)[0]
for x, weight in enumerate(w):
    if weight == 0:
        Lambda[x,x] = 1
        w[x] = 1
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# number of iterations
n_steps = 100

states = np.zeros(n_steps, dtype=int)
# initial configuration
x = np.random.choice(actions, size = n_players)
x_id = np.ravel_multi_index(tuple(x), dims = n_states)
states[0] = x_id

transition_times = np.zeros(n_steps)
t_next = -np.log(np.random.rand())/w[0]

for i in range(1,n_steps):
    states[i] = np.random.choice(n_config, p=P[states[i-1],:])
    transition_times[i] = transition_times[i-1] + t_next
    t_next = -np.log(np.random.rand())/w[states[i]]

In [ ]:
# Compute and plot the evolution of the potential along the simulated 
# path of the chain

# store the potential value at each step of the simulation
potentials = np.zeros(n_steps)
for step in range(n_steps):
    x_id = states[step]
    x = np.array(np.unravel_index(x_id, shape = n_states))
    for i, j in G.edges:
        potentials[step] += pot[x[i],x[j]]

plt.step(transition_times, potentials, 'bo-', where="post");

In [ ]:
# To better understand the process, visualize the 
# evolution of the game's configuration in time

import matplotlib.pyplot as plt

# Plot the evolution of the game's configurations
fig = plt.figure(figsize=(9,12))
for t in range(0,min(n_steps,25)):
    plt.subplot(5,5,t+1)
    x_id = states[t]
    x = np.array(np.unravel_index(x_id, shape = n_states))
    nx.draw(G,
        pos = pos,
        with_labels=True,
        nodelist=np.argwhere(x==0).T[0].tolist(),
        node_color = 'r')
    nx.draw(G,
        pos = pos,
        with_labels=True,
        nodelist=np.argwhere(x==1).T[0].tolist(),
        node_color = 'b')
    plt.title('jump step = {0}'.format(t+1))

**A few observations**

- Even if the noisy best response dynamics falls in the global maximum configuration (all zeros), still it will escape from this configuration, for any value of $\eta$, because the dynamics is ergodic. This is a crucial observation, which distinguishes the noisy best response from the best response.

- Also the 'all ones' configuration is a Nash equilibrium, and thus a local maximum of the potential. If $\eta$ is large and the system falls in the local maximum, it takes a lot to escape this configuration. This is due to the fact that the dynamics becomes "less ergodic" as $\eta$ grows large, in the sense that it takes a lot to escape from Nash equilibria, and in general it takes a lot to descend the potential, because the players become more rational as $\eta$ grows.

- In mathematical terms, this statement may be rephrased as follows: "as $\eta$ grows large, the invariant distribution of the dynamics tends to concentrate most of the mass on the maximizers of the potential, but it takes more time to relax to the invariant distribution." This is also related to the fact that the transition graphs becomes less connected as $\eta$ grows (because the probability of an agent choosing a suboptimal strategy tends to $0$)

It is instructive to simulate the dynamics from the Nash equilibrium 'all ones' with a large value of $\eta$.

In [ ]:
# Define the network coordination game

# number of nodes in G = number of players in the game
n_players = len(G)
# utility values
a = 1
b = 1/2
c = 0
d = 0
# utility matrix for the 2x2 coordination game
phi = np.array([[a,d],
                [c,b]])
# the potential function of the 2x2 coordination game
pot = np.array([[a-c,0],
                [0,b-d]])
# inverse noise parameter
eta = 10
# available actions
actions = [0,1]
n_actions = len(actions)
# adjacency matrix
W = nx.convert_matrix.to_numpy_matrix(G)

def utility(player, x, phi):
    result = 0
    for other_player in G.neighbors(player):
         result += phi[x[player], x[other_player]]
    return result

In [ ]:
# Define the noisy best response dynamics

# Initialize transition rates matrix
n_config = n_actions**n_players
Lambda = sp.sparse.lil_matrix((n_config,n_config))

# Number of actions for each player
n_states =tuple(n_actions for _ in range(n_players))

# Fill transition rates matrix
for x_id in range(n_config):
    x = np.unravel_index(x_id,shape=n_states)
    x = np.array(x)
    for player in range(n_players):
        # compute utilities gained by `player` for each of its possible actions
        # while the other players are in the current configuration x
        utilities = np.zeros(n_actions)
        for action in actions:
            y = np.array(x)
            y[player] = action
            utilities[action] = utility(player,y,phi)
        exp_utilities = np.exp(eta*utilities)
        for action in actions:
            if action == x[player]:
                continue
            y = np.array(x)
            y[player] = action
            y_id = np.ravel_multi_index(tuple(y), dims = n_states)
            Lambda[x_id, y_id] += exp_utilities[action] / np.sum(exp_utilities)

In [ ]:
# Simulate the continuous time Markov chain with 2nd approach: local clocks

w = np.sum(Lambda, axis=1)
w = np.array(w.T)[0]
for x, weight in enumerate(w):
    if weight == 0:
        Lambda[x,x] = 1
        w[x] = 1
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# number of iterations
n_steps = 25

# initial configuration all ones
x_id = n_config-1
states[0] = x_id

transition_times = np.zeros(n_steps)
t_next = -np.log(np.random.rand())/w[0]

P_cum = np.cumsum(P, axis=1)

for i in range(1,n_steps):
    states[i] = np.random.choice(n_config, p=P[states[i-1],:])
    transition_times[i] = transition_times[i-1] + t_next
    t_next = -np.log(np.random.rand())/w[states[i]]

In [ ]:
# Compute and plot the evolution of the potential along the simulated 
# path of the chain

# store the potential value at each step of the simulation
potentials = np.zeros(n_steps)
for step in range(n_steps):
    x_id = states[step]
    x = np.array(np.unravel_index(x_id, shape = n_states))
    for i, j in G.edges:
        potentials[step] += pot[x[i],x[j]]

plt.step(transition_times, potentials, 'bo-', where="post");

In [ ]:
# To better understand the process, visualize the 
# evolution of the game's configuration in time

import matplotlib.pyplot as plt

# Plot the evolution of the game's configurations
fig = plt.figure(figsize=(9,12))
for t in range(0,min(n_steps,25)):
    plt.subplot(5,5,t+1)
    x_id = states[t]
    x = np.array(np.unravel_index(x_id, shape = n_states))
    nx.draw(G,
        pos = pos,
        with_labels=True,
        nodelist=np.argwhere(x==0).T[0].tolist(),
        node_color = 'r')
    nx.draw(G,
        pos = pos,
        with_labels=True,
        nodelist=np.argwhere(x==1).T[0].tolist(),
        node_color = 'b')
    plt.title('jump step = {0}'.format(t+1))

Observe that for a very long time interval the dynamics does not escape from the 'all ones' Nash equilibrium. However, as the simulation is observed for even longer times, the time spent in the 'all ones' configuration is expected to be a negligible fraction of the total time (as shown in the next step). This shows that for large $\eta$ it takes a lot to escape from local maxima and to relax to the invariant distribution $\pi$. If one considers instead the BR dynamics, the 'all ones' configuration belongs to $\mathcal{N}_\infty$ and the dynamics does not escape from that configuration.

#### Step 3
> If the simulation continues for a long time, will the probability to be in the different configurations converge towards some specific values? In that case, how can these be computed?

According to the theory about noisy best response dynamics, this type of dynamics is, for any choice of $\eta > 0$, an irreducible reversible Markov chain with invariant probability distribution
$$
\pi_x = \frac{e^{\eta \Phi(x)}}{Z_\eta}, \quad Z_\eta = \sum_{y\in \mathcal{X}} e^{\eta \Phi(y)}.
\tag{1}
$$

Compute the values of $\pi$.

In [ ]:
# TO DO

#### Step 4
> Compute the ratio between the probability that after a long time (i.e., in stationarity) all nodes choose action 1 and the probability that all nodes choose action 0, both for case **1** and case **2**. Does this seem to agree with your simulations?

According to (1), we have that 
$$
\frac{\pi_{(1,1,1,1,1)}}{\pi_{(0,0,0,0,0)}} = \frac{e^{\eta \Phi(1,1,1,1,1)}}{e^{\eta \Phi(0,0,0,0,0)}}.
$$
We can thus compute the result according to the following code: 

In [ ]:
print("Case 1:")
# utility values
a = 1
b = 1
c = 0
d = 0
# the potential function of the 2x2 game
eta = 2
pot = np.array([[a-c,0],
                [0,b-d]])
x_0 = [0 for node in G]
potential_0 = 0
for i, j in G.edges:
    potential_0 += 0.5*pot[x_0[i],x_0[j]]

x_1 = [1 for node in G]
potential_1 = 0
for i, j in G.edges:
    potential_1 += 0.5*pot[x_1[i],x_1[j]]

probability_ratio_1_over_0 = np.exp(eta*potential_1) / np.exp(eta*potential_0)
print("The ratio between the probability of configuration 'all ones' and configuration 'all zeros' is", probability_ratio_1_over_0, "\n")

print("Case 2:")

# utility values
a = 1
b = 1/2
c = 0
d = 0
# the potential function of the 2x2 game
pot = np.array([[a-c,0],
                [0,b-d]])

x_0 = [0 for node in G]
potential_0 = 0
for i, j in G.edges:
    potential_0 += pot[x_0[i],x_0[j]]

x_1 = [1 for node in G]
potential_1 = 0
for i, j in G.edges:
    potential_1 += pot[x_1[i],x_1[j]]

probability_ratio_1_over_0 = np.exp(eta*potential_1) / np.exp(eta*potential_0)
print("The ratio between the probability of configuration 'all ones' and configuration 'all zeros' is", probability_ratio_1_over_0)


Check if this result agrees with your simulations by estimating $\pi$ for case **1** and **2** by measuring the time spent in each configuration. Let's do a long simulation.

In [ ]:
# number of nodes in G = number of players in the game
n_players = len(G)
# utility values
a = 1
b = 1/2
c = 0
d = 0
# utility matrix for the 2x2 coordination game
phi = np.array([[a,d],
                [c,b]])
# the potential function of the 2x2 coordination game
pot = np.array([[a-c,0],
                [0,b-d]])
# inverse noise parameter
eta = 2
# available actions
actions = [0,1]
n_actions = len(actions)
# adjacency matrix
W = nx.convert_matrix.to_numpy_matrix(G)

def utility(player, x, phi):
    result = 0
    for other_player in G.neighbors(player):
         result += phi[x[player], x[other_player]]
    return result

In [ ]:
# Define the noisy best response dynamics

# Initialize transition rates matrix
n_config = n_actions**n_players
Lambda = sp.sparse.lil_matrix((n_config,n_config))

# Number of actions for each player
n_states =tuple(n_actions for _ in range(n_players))

# Fill transition rates matrix
for x_id in range(n_config):
    x = np.unravel_index(x_id,shape=n_states)
    x = np.array(x)
    for player in range(n_players):
        # compute utilities gained by `player` for each of its possible actions
        # while the other players are in the current configuration x
        utilities = np.zeros(n_actions)
        for action in actions:
            y = np.array(x)
            y[player] = action
            utilities[action] = utility(player,y,phi)
        exp_utilities = np.exp(eta*utilities)
        for action in actions:
            if action == x[player]:
                continue
            y = np.array(x)
            y[player] = action
            y_id = np.ravel_multi_index(tuple(y), dims = n_states)
            Lambda[x_id, y_id] += exp_utilities[action] / np.sum(exp_utilities)

In [ ]:
#Simulate the continuous time Markov chain with 2nd approach: local clocks

w = np.sum(Lambda, axis=1)
w = np.array(w.T)[0]
for x, weight in enumerate(w):
    if weight == 0:
        Lambda[x,x] = 1
        w[x] = 1
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# number of iterations
n_steps = 100000

states = np.zeros(n_steps, dtype=int)
# initial configuration
x = np.random.choice(actions, size = n_players)
x_id = np.ravel_multi_index(tuple(x), dims = n_states)
states[0] = x_id

transition_times = np.zeros(n_steps)
t_next = -np.log(np.random.rand())/w[0]

P_cum = np.cumsum(P, axis=1)

for i in range(1,n_steps):
    states[i] = np.random.choice(n_config, p=P[states[i-1],:])
    transition_times[i] = transition_times[i-1] + t_next
    t_next = -np.log(np.random.rand())/w[states[i]]

In [ ]:
# compute frequencies
intervals = np.diff(transition_times, n=1, append = transition_times[-1] + t_next)
frequencies = np.zeros(n_config)

for node in range(n_config):
    visits = np.argwhere(states == node)
    frequencies[node] = np.sum(intervals[visits])/(transition_times[-1] + t_next)
print("Empirical frequencies:", frequencies, "\n")

print("Fraction of time spent in configuration", np.unravel_index(n_config-1, shape = n_states), "/ Fraction of time spent in configuration", np.unravel_index(0, shape = n_states), ":", frequencies[-1]/frequencies[0])

#### Step 5. 
> Compute the ratio between the probability that after a long time (i.e., in stationarity) one node chooses action 1 and the rest choose action 0, and the probability that all nodes choose action 0, both for case **1** and case **2**.

Using the invariant probability distribution formula (1) again, we can compute the ratio

$$
\frac{\sum_{i \in \mathcal V} \pi_{\delta(i)}}{\pi_{(0,0,0,0,0)}}
$$

where $\delta^{(i)}_i=1$ and $\delta^{(i)}_j=0$, $\forall j \neq i$.

In [ ]:
print("Case 1:")
# utility values
a = 1
b = 1
c = 0
d = 0
# the potential function of the 2x2 game
pot = np.array([[a-c,0],
                [0,b-d]])
x_0 = [0 for node in G]
potential_0 = 0
for i, j in G.edges:
    potential_0 += 0.5*pot[x_0[i],x_0[j]]

exp_potential_1 = 0
for player in G:
    x_1 = [0 for node in G]
    x_1[player] = 1
    potential_1 = 0
    for i, j in G.edges:
        potential_1 += 0.5*pot[x_1[i],x_1[j]]
    exp_potential_1 += np.exp(eta*potential_1)

# print(exp_potential_1)
probability_ratio_1_over_0 = exp_potential_1 / np.exp(eta*potential_0)
print("The ratio between probability of configuration 'one player playing 1, rest playing 0' and configuration (0,0,0,0,0) is", probability_ratio_1_over_0, "\n")

print("Case 2:")

# utility values
a = 1
b = 1/2
c = 0
d = 0
# the potential function of the 2x2 game
pot = np.array([[a-c,0],
                [0,b-d]])

x_0 = [0 for node in G]
potential_0 = 0
for i, j in G.edges:
    potential_0 += 0.5*pot[x_0[i],x_0[j]]

exp_potential_1 = 0
for player in G:
    x_1 = [0 for node in G]
    x_1[player] = 1
    potential_1 = 0
    for i, j in G.edges:
        potential_1 += 0.5*pot[x_1[i],x_1[j]]
    exp_potential_1 += np.exp(eta*potential_1)

# print(exp_potential_1)
probability_ratio_1_over_0 = exp_potential_1 / np.exp(eta*potential_0)
print("The ratio between probability of configuration 'one player playing 1, rest playing 0' and configuration (0,0,0,0,0) is", probability_ratio_1_over_0)


Even if in case 1 and 2 are different, the ratio 

$$
\frac{\sum_{i \in \mathcal V} \pi_{\delta^{(i)}}}{\pi_{(0,0,0,0,0)}}
$$

is the same in the two cases.
This happens because the potential value of configurations $\delta^(i)$ and $(0,0,0,0,0)$ does not depends on $b$, as the configurations does not contain any couple of neighboring players playing action $1$.

## On the difference between BR and NBR (in potential games)

- The NBR dynamics is irreducible, the BR is not (transitions are allowed only in the increasing direction of the potential).
- The NBR dynamics admits a unique invariant distribution. The BR converges almost surely to the set of sink components of Nash equilibria (called recurrent equilibria, and indicated by $\mathcal{N}_\infty$). However, $\mathcal{N}_\infty$ may in general contain multiple disconnected sets of Nash equilibria, and which one is selected depends on the initial condition and on the realization of the Markov process.
- As $\eta \to \infty$, the NBR dynamics for long times condensate its probability distribution to the global maximizers of the potential $\Phi$. However, $\mathcal{N}_\infty$ does not in general coincide with the global maximizers of the potential. The only property that can be proven is that $\mathcal{N}_\infty$ contains the global maxima of $\Phi$.

In the previous example, with $a=1, b=1/2$, both $x = \mathbf{1}$ and $x=\mathbf{0}$ are consensus states and Nash equilibria of the game. Both of them belong to $\mathcal{N}_\infty$. Indeed, if the configuration is $(1,1,1,1,1)$ or $(0,0,0,0,0)$ no rational agents modify their strategy, and the BR dynamics gets stucked in that configuration. However, $\Phi(1,1,1,1,1)<\Phi(0,0,0,0,0)$, thus the NBR dynamics with $\eta \to \infty$ for long times (after relaxating to the invariant distribution $\pi$) will spent almost fraction $1$ of the total time in $(0,0,0,0,0)$. Observe that this does not mean that the configuration $(0,0,0,0,0)$ is trapping for the NBR!!! 